![](../docs/banner.png)

# Laboratorium 3: Tworzenie modeli uczenia maszynowego

## Efekty kształcenia laboratorium
---

Podczas niniejszych zajęć:

+ poznasz podstawowe pojęcia z zakresu uczenia maszynowego
+ dowiesz się jak przeprowadzić analizę statystyczną zbioru danych i na co zwrócić uwagę
+ poznasz i przeprowadzisz cały proces uczenia masznowego - od wczytania zbioru, poprzez jego transformacje, uczenie, aż po walidację krzyżową i dostrajanie parametrów modelu
+ poznasz podstawowe mechanizmy sprawdzania jakości modelu uczenia maszynowego

## Wprowadzenie
---

1. Uczenie maszynowe (ang. _machine learning_) jest obecnie najpopularniejszą dziedziną sztucznej inteligencji. 
2. Polega ono na automatycznej budowie _modelu_ poprzez ekspozycję _algorytmu_ na _dane treningowe_ w procesie zwanym _uczeniem_. 
3. Model uczenia maszynowego posiada zdolność rozpoznawania wzorców wykrytych w danych, dzięki czemu jest w stanie dokonywać _predykcji_. 
4. Celem uczenia maszynowego jest tworzenie modeli zdolnych do _generalizacji_, tj. poprawnego predykowania na danych nie użytych do treningu. 

W zależności od problemu który ma zostać rozwiązany, stosuje się różne metody i algorytmy, w szczególności:
- jeśli celem jest przypisanie danym pewnej kategorycznej etykiety, mówimy o *klasyfikacji* - np. określenie czy na zdjęciu jest kot, czy pies
- jesli celem jest umiejscowienie danych na pewnej ciągłej skali liczbowej, mówimy o *regresji* - np. prognozowanie wartości produktu
- jeśli celem jest zgrupowanie podobnych sobie danych, mówimy o *klasteryzacji* - np. przypisanie użytkowników Twittera do grup w zależności od poruszanych tematów

Klasyfikacja i regresja są przykładami _uczenia nadzorowanego_, gdzie do treningu oprócz danych wejściowych opisujących problem, musimy też posiadać _etykietę_, tj. klasę lub wartość która ma być wyjściem modelu.

Klasteryzacja to przykład _uczenia nienadzorowanego_, gdzie model realizuje swoje zadanie bez dodatkowych informacji z naszej strony. 

![](../docs/rodzaje_uczenia.png)

## Analiza statystyczna zbioru treningowego
---

Przed rozpoczęciem rozwiązywania problemu przy użyciu metod uczenia maszynowego, w szczególności przed rozpoczęciem budowania modelu, konieczne jest sprawdzenie, z jakimi danymi przyszło się nam mierzyć. 

Wśród podstawowych kwestii, które powinniśmy sprawdzić, są:
- ile mamy cech?
- które spośród nich to cechy kategoryczne, a które numeryczne?
- jakie wartości przyjmują poszczególne cechy?
- czy wśród danych są brakujące wartości?
- czy istnieje i jak wygląda etykieta? (w szczególności - czy mierzymy się z zadaniem klasyfikacji, regresji czy klasteryzacji?) 
- czy dane są zbalansowane względem danej wyjściowej?

Dla małych i prostych zbiorów do nauki (tzw. _toy tasks_), zazwyczaj wystarczające jest ręczne przejrzenie pliku z danymi, by potrafić odpowiedzieć na w/w pytania. Niemniej przy bardziej ambitnych zadaniach, z pomocą przychodzą narzędzia automatyzujące pracę. 

### Przykładowy zbiór danych

Przeanalizujmy klasyczny zbiór danych dotyczący wina (opublikowany przez `Forina, M. et al, PARVUS - An Extendible Package for Data Exploration, Classification and Correlation. Institute of Pharmaceutical and Food Analysis and Technologies, Via Brigata Salerno, 16147 Genoa, Italy`, więcej informacji [tutaj](https://archive.ics.uci.edu/ml/datasets/wine))

Zbiór zawiera właściwości fizykochemiczne różnych próbek wina pobranych z jednego z regionów słonecznej Italii, jednakże pochodzących od trzech różnych plantatorów. Założeniem problemu jest określenie, który z nich jest wytwórcą danej próbki.

Zacznijmy od wczytania zbioru:

In [13]:
from sklearn.datasets import load_wine

dataset, y = load_wine(as_frame=True, return_X_y=True)
dataset['target'] = y
dataset.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


Na pierwszy rzut oka możemy stwierdzić, że wszystkie kolumny są numeryczne, ale ich wartości różnią się dość znacząco. 

Podstawowe informacje o statystykach zbioru danych możemy uzyskać przy wbudowanej w Pandas metodzie `describe()`

In [14]:
dataset.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258,0.938202
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,0.775035
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000,0.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000,0.000000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000,1.000000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000,2.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000,2.000000


Jednakże w celu dokładniejszej analizy zbioru, posłużymy się biblioteką Pandas Profiling.

### Pandas Profiling

Jest to biblioteka automatycznie analizująca zbiór danych i generujaca interaktywny raport. Alternatywnie, raport można zapisać w formacie `.html`

Na dzień tworzenia tego zadania (2021.02.06) wersja 2.10 biblioteki zawiera błąd nie pozwalający generować raportu w środowisku Jupyter. Posłużymy się więc wersją 2.9, która działa poprawnie.

Upewnijmy się, że jest ona poprawnie zainstalowana:

In [15]:
!pip install pandas-profiling==2.9.0

Użycie biblioteki jest niezwykle proste:

In [16]:
from pandas_profiling import ProfileReport

profile = ProfileReport(dataset)
profile.to_widgets()

Summarize dataset:   0%|          | 0/28 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Z raportu dowiadujemy się między innymi:
- mamy 13 kolumn numerycznych (dane wejściowe), jedną kategoryczną (etykieta) - będziemy więc zajmować się klasyfikacją
- klasy są całkiem nieźle zbalansowane (39%, 33%, 27%)
- nie mamy brakujących wartości ani duplikatów
- możemy dokładnie przeanalizować statystyki poszczególnych cech, ich histogramy oraz wykresy zależności pomiędzy nimi

## Uczenie maszynowe przy pomocy sklearn
---

Scikit-learn (aka sklearn) jest aktualnie najpopularniejszą biblioteką w Pythonie pozwalającą kompleksowo przeprowadzać proces machine learningowy, który zazwyczaj składa się z następujących elementów:

1. wczytanie zbioru danych
2. preprocessing zbioru:
    - transformacje cech (normalizacja, skalowanie, kodowanie, dyskretyzacja, embeddowanie, ekstrakcja cech itd)
    - rozwiązanie kwestii brakujących i powtarzających się danych
    - rozwiązanie kwestii imbalansu klas (oversampling/undersampling)
    - augmentacja danych
3. selekcja cech/redukcja wymiarowości
4. uczenie modelu 
5. dostosowywanie (fine-tuning) parametrów

Jednakże zanim pokrótce zapoznamy się z tymi metodami, upewnijmy się, że scikit-learn jest poprawnie zainstalowany:

In [17]:
!pip install scikit-learn

### Wczytanie zbioru danych

Scikit-learn posiada wbudowany zestaw standardowych, benchmarkowych zbiorów danych - jak np. używany w przykładzie zbiór _wine_. Pozwala także w prosty sposób generować syntetyczne dane.

Więcej informacji o dostępnych w bibliotece zbiorach danych [tutaj](https://scikit-learn.org/stable/datasets.html)

W przypadku rzeczywistych problemów i własnych datasetów - będziemy jednak używać biblioteki Pandas.

In [18]:
# użyty już przykład wczytania datasetu wine, dla przypomnienia

from sklearn.datasets import load_wine

X, y = load_wine(as_frame=True, return_X_y=True)
X.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


Trzymając się przyjętej konwencji matematycznej, zbiór danych wejściowych nazywamy `X`, natomiast wyjściowych - `y`

### Preprocessing zbioru danych

Odpowiednie przygotowanie danych uczących jest kluczem do sukcesu każdego przedsięwzięcia machine learningu - w myśl zasady `Garbage in, garbage out`. Przed przystąpieniem do uczenia, zazwyczaj konieczne jest wykonanie co najmniej kilku kroków wstępnego przetwarzania danych. 

Krok ten silnie uzależniony jest od samych danych - inaczej przygotowywać będziemy dane tekstowe, numeryczne, dźwiękowe czy obrazy. Dużą rolę gra tutaj także planowany do użycia algorytm - część z nich wymaga np. ustandaryzowanych danych (tj. pochodzących z rozkładu zbliżonego do normalnego, posiadających średnia w 0 i odchylenie standardowe równe 1). 

Duże znaczenie dla jakości procesu ma balans klas - w przypadku danych niezbalansowanych, część algorytmów wykazuje tendencje do preferowania klasy nadreprezentowanej, przez co popełniają błędy.

Ponadto w wielu zbiorach danych występują cechy, które nie są informacyjne. Usunięcie takich cech często pozwala na poprawę jakości predykcji. Ograniczenie ilości cech ułatwia także technicznie proces uczenia - algorytm musi przetworzyc mniej danych, wiec dzieje się to szybciej. Istnieje wiele metod selekcji cech, np. bazujacych na testach statystycznych. Ich ogólna idea polega na ocenieniu cech wg. zadanej miary jakości, a następnie wyboru najbardziej wartościowych. 

Alternatywnie stosuje się metody redukcji wymiarowości. W przeciwieństwie do selekcji cech, która jedynie wybiera spośród istniejących już danych pewien ich podzbiór, algorytmy redukcji wymiarowości przekształcają dane, tworząc nowe, bardziej informacyjne cechy. Najpopularniejszą metodą redukcji wymiarowości jest `PCA`

``` {hint}
Preprocessingowi poświęcone zostało Laboratorium 4, gdzie w/w mechanizmy zostaną omówione dokładniej. 
```

W naszym komfortowym przykładzie nie ma dużej potrzeby dostosowywania danych:
- klasy są w miarę dobrze zbalansowane
- nie ma wartości brakujących
- nie ma duplikatów
- nie ma danych kategorycznych, poza etykietą, więc nie ma potrzeby ich kodowania

Operujemy natomiast na danych numerycznych, które mają skrajnie różne wartości pomiędzy cechami, poddamy je więc standaryzacji

In [19]:
from sklearn.preprocessing import StandardScaler

# najpierw przekształćmy dataset do formatu numpy, akceptowalnego przez scikit-learn
X = X.to_numpy()

In [20]:
# zdefiniujmy funkcję wyświetlajacą statystyki średniej i odchylenia standardowego
def print_dataset_stats(X):
    print('Średnia cech:\n', X.mean(axis=0))
    print()
    print('Odchylenie standardowe cech:\n', X.std(axis=0))
    print()

```{important}
Większość algorytmów i klas zawartych w scikit-learn implementuje interfejs fit/transform - tj. przy pomocy funkcji `fit` dokonywane jest dopasowywanie algorytmów do danych uczących (uczenie/ustalanie parametrów modelu), natomiast przy pomocy funkcji `transform` dokonywane jest przekształcenie danych.
```

In [21]:
scaler = StandardScaler().fit(X)

print("Przed standaryzacją")
print_dataset_stats(X)
X = scaler.transform(X)
print("Po standaryzacji")
print_dataset_stats(X)

Przed standaryzacją
Średnia cech:
 [1.30006180e+01 2.33634831e+00 2.36651685e+00 1.94949438e+01
 9.97415730e+01 2.29511236e+00 2.02926966e+00 3.61853933e-01
 1.59089888e+00 5.05808988e+00 9.57449438e-01 2.61168539e+00
 7.46893258e+02]

Odchylenie standardowe cech:
 [8.09542915e-01 1.11400363e+00 2.73572294e-01 3.33016976e+00
 1.42423077e+01 6.24090564e-01 9.96048950e-01 1.24103260e-01
 5.70748849e-01 2.31176466e+00 2.27928607e-01 7.07993265e-01
 3.14021657e+02]

Po standaryzacji
Średnia cech:
 [-8.38280756e-16 -1.19754394e-16 -8.37033314e-16 -3.99181312e-17
 -3.99181312e-17  0.00000000e+00 -3.99181312e-16  3.59263181e-16
 -1.19754394e-16  2.49488320e-17  1.99590656e-16  3.19345050e-16
 -1.59672525e-16]

Odchylenie standardowe cech:
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]



### Uczenie modelu

Mając odpowiednio przygotowane dane, możemy przystąpić do procesu uczenia. Pierwszym krokiem będzie podział zbioru na część treningową (na której nauczymy model) oraz testową (na której przetestujemy jego jakość). Taki podział pozwala zmierzyć zdolność modelu do generalizacji - sprawdzamy jakość na danych, które nie były użyte do nauki, których "model wcześniej nie widział".

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [23]:
print('X_train: ', X_train.shape)
print('X_test: ', X_test.shape)
print('y_train: ', y_train.shape)
print('y_test: ', y_test.shape)

X_train:  (142, 13)
X_test:  (36, 13)
y_train:  (142,)
y_test:  (36,)


Finalnie jesteśmy gotowi stworzyć klasyfikator. Dla przykładu użyjemy algorytmu SVM. Przykłady innych modeli można znaleźć [tutaj](https://scikit-learn.org/stable/supervised_learning.html)

In [24]:
from sklearn.svm import SVC
classifier = SVC()
classifier.fit(X_train, y_train)

SVC()

Możemy go teraz użyć w celu predykcji nowych wartości:

In [25]:
classifier.predict(X_test[:2])

array([0, 2])

```{important}
Istnieje wiele modeli uczenia maszynowego. Wybór odpowiedniego zależy od problemu który rozwiązujemy, danych które posiadamy, pożądanemu rezultatowi i warunkom działania (np. pożądana szybkość uczenia i inferencji).

Do najpopularniejszych modeli nadzorowanych należą - drzewa decyzyjne (i lasy losowe), kNN, SVM, Naive Bayes, sieci neuronowe.

Niestety, dokładne wprowadzenie w fascynującą dziedzinę modelu uczenia maszynowego leży poza zakresem tego laboratorium.

```

### Potoki

W powyższym przykładzie po wczytaniu danych przetworzyliśmy je przez dwa algorytmy - `StandardScaler` i `SVM`. W przypadku bardziej rozbudowanego preprocessingu, niezwykle przydatna staje się możliwość łączenia wszystkich użytych mechanizmów w jeden potok:

In [26]:
from sklearn.pipeline import make_pipeline

X, y = load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

pipeline_classifier = make_pipeline(
    StandardScaler(), 
    SVC()
)

pipeline_classifier.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC())])

In [27]:
pipeline_classifier.predict(X_test[:2])

array([0, 1])

### Analiza jakości modelu

Na tym etapie posiadamy zbiór danych podzielony na część treningową i testową, oraz klasyfikator nauczony na częsci treningowej. Jak sprawdzić, jak dobrze jest on nauczony?

Do określenia jakości modelu służy szereg metryk, różnych w zależności od problemu. 

W zadaniu klasyfikacji, standardowo używa się następujących miar wywodzących się z klasyfikacji binarnej:
- accuracy, oznaczająca procent poprawnie oznaczonych przykładów
- F1 - będąca średnią miar `precision` i `recall`, mierzących skłonności modelu do niepopełniania błędów

In [46]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

y_predicted = pipeline_classifier.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_predicted))
print('F1: ', f1_score(y_test, y_predicted, average='macro'))

Accuracy:  0.9722222222222222
F1:  0.9747545582047685


Dokładniejsze informacje o jakości klasyfikacji możemy uzyskac np. przy pomocy metody `classification_report`

In [47]:
from sklearn.metrics import classification_report

classification_report(y_test, y_predicted, output_dict=True)

{'0': {'precision': 1.0,
  'recall': 0.9166666666666666,
  'f1-score': 0.9565217391304348,
  'support': 12},
 '1': {'precision': 0.9375,
  'recall': 1.0,
  'f1-score': 0.967741935483871,
  'support': 15},
 '2': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 9},
 'accuracy': 0.9722222222222222,
 'macro avg': {'precision': 0.9791666666666666,
  'recall': 0.9722222222222222,
  'f1-score': 0.9747545582047685,
  'support': 36},
 'weighted avg': {'precision': 0.9739583333333334,
  'recall': 0.9722222222222222,
  'f1-score': 0.9720663861617579,
  'support': 36}}

W przypadku zadania regresji, standardowo używaną miarą jest błąd średniokwadratowy (`ang. mean square error`). Więcej informacji o różnych rodzajach metryk jakości modelu - [tutaj](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)

### Sprawdzanie krzyżowe

Powyżej nauczyliśmy model klasyfikować - i to z bardzo dobrą jakością! 

Jednakże, nauczyliśmy nasz model na pewnym podzbiorze danych i przetestowaliśmy na innym. Nie mamy pewności, czy model sam w sobie wykazuje się dobrą jakością, czy to akurat ten specyficzny wybór podzbiorów danych daje tak dobre wyniki.

Z pomocą przychodzi tutaj `sprawdzanie krzyżowe` `(ang. cross-validation)`. Jest to procedura polegająca na kilkukrotnym uruchomieniu procedury uczenia modelu, za każdym razem na innym podzbiorze danych. Uśredniony wynik takich kilku modeli pozwala z większą dokładnością wyrokować o jakości samego modelu. 

In [29]:
from sklearn.model_selection import cross_validate

X, y = load_wine(return_X_y=True)
cv_results = cross_validate(
    pipeline_classifier, 
    X, 
    y,
    scoring='f1_macro'
)

In [30]:
cv_results['test_score'].mean()

0.983504493924021






### Dostosowywanie parametrów

Praktycznie każdy z algorytmów uczenia maszynowego posiada pewne parametry (lub hiper-parametry), które wpływają na proces uczenia. W przypadku użytego algorytmu SVM są to np. rodzaj funkcji jądra i stopień regularyzacji. 

Odpowiednie dostosowanie parametrów algorytmu do danego problemu może znacząco poprawić jakość jego działania. 

Sklearn posiada zaimplementowanych kilka strategii poszukiwania najlepszych parametrów. Użyjemy tutaj najprostszego podejścia losowego:

In [31]:
from sklearn.model_selection import RandomizedSearchCV

X, y = load_wine(return_X_y=True)

param_distributions = {'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                       'svc__C': [0.1, 0.5, 1, 2, 4]}

# now create a searchCV object and fit it to the data
search = RandomizedSearchCV(estimator=pipeline_classifier,
                            n_iter=5,
                            param_distributions=param_distributions)
search.fit(X, y)

print(search.best_params_)

{'svc__kernel': 'linear', 'svc__C': 0.1}


Mechanizm wykonuje ilosc powtórzeń zadaną przez argument `n_iter` i losuje za każdym razem wartości parametrów spośród podanych. Dla każdego z nich wykonuje cross-walidację. Możemy też przeanalizować dokładne wyniki tego procesu:

In [32]:
search.cv_results_

{'mean_fit_time': array([0.00537829, 0.00644789, 0.00556135, 0.00395641, 0.00475116]),
 'std_fit_time': array([0.00302507, 0.00417539, 0.00360711, 0.0018147 , 0.00253776]),
 'mean_score_time': array([0.00384507, 0.00235186, 0.00214186, 0.00119901, 0.00100121]),
 'std_score_time': array([0.00432281, 0.00146971, 0.00099259, 0.00075927, 0.00032661]),
 'param_svc__kernel': masked_array(data=['rbf', 'linear', 'linear', 'linear', 'poly'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_svc__C': masked_array(data=[0.1, 2, 0.1, 4, 2],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'svc__kernel': 'rbf', 'svc__C': 0.1},
  {'svc__kernel': 'linear', 'svc__C': 2},
  {'svc__kernel': 'linear', 'svc__C': 0.1},
  {'svc__kernel': 'linear', 'svc__C': 4},
  {'svc__kernel': 'poly', 'svc__C': 2}],
 'split0_test_score': array([0.97222222, 0.94444444, 0.97222222, 0.9444

Parametry modelu sprawdzić możemy przy pomocy metody `get_params()` lub [w dokumentacji](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)

In [49]:
pipeline_classifier.get_params()

{'memory': None,
 'steps': [('standardscaler', StandardScaler()), ('svc', SVC())],
 'verbose': False,
 'standardscaler': StandardScaler(),
 'svc': SVC(),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'svc__C': 1.0,
 'svc__break_ties': False,
 'svc__cache_size': 200,
 'svc__class_weight': None,
 'svc__coef0': 0.0,
 'svc__decision_function_shape': 'ovr',
 'svc__degree': 3,
 'svc__gamma': 'scale',
 'svc__kernel': 'rbf',
 'svc__max_iter': -1,
 'svc__probability': False,
 'svc__random_state': None,
 'svc__shrinking': True,
 'svc__tol': 0.001,
 'svc__verbose': False}

## Zadania
---

1\. (1 pkt) Przy pomocy sklearn, wczytaj zbiór danych [California Housing](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset) i wyświetl jego przykładowe kilka rekordów

In [1]:
#TUTAJ UMIEŚĆ KOD

2\. (1.5 pkt) Przy pomocy biblioteki Pandas Profiling przeanalizuj w/w zbiór i odpowiedz na następujące pytania:

- ile znajduje się w zbiorze cech kategorycznych, a ile numerycznych? 
- czy zmienna wyjściowa jest kategoryczna, czy numeryczna?
- czy rozpatrujemy problem klasyfikacji, klasteryzacji czy regresji?
- ile w zbiorze jest brakujących wartości?
- czy któraś z cech koreluje ze zmienną wyjściową? Jeśli tak - która?


In [2]:
# TUTAJ UMIEŚC ODPOWIEDZI

3\. (2 pkt) stwórz pipeline składający się z modułu standaryzacji i estymatora - w zależności czy problem dotyczy klasyfikacji czy regresji, użyj [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) lub [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) 

In [3]:
# TUTAJ UMIEŚĆ KOD

4\. (4 pkt) używając sprawdzianu krzyżowego z 10 foldami (sprawdź odpowiedni parametr w [dokumentacji](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)) sprawdź jakość predykcji, używając odpowiednich do zadania metryk (wybierz odpowiednią spośród tych [tutaj](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics))

In [4]:
# TUTAJ UMIEŚĆ KOD

5\. (3 pkt) Używając cross-walidacji, porównaj jakość modelu używającego i nieużywającego standaryzacji cech.

In [5]:
# TUTAJ UMIEŚC KOD

6\. (3.5 pkt) Używając metody [GridSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) sprawdź, jakie parametry modelu najlepiej pasują do rozwiązywanego problemu

```{important}
Pamiętaj, że mechanizm GridSearch przeszukuje wszystkie możliwe kombinacje podanych parametrów. Nie podawaj ich zbyt wiele, by skończyć zadanie w sensownym czasie.
```

In [38]:
# TUTAJ UMIEŚĆ KOD